In [1]:
# from prepare_data import create_training_data

# create_training_data()

In [2]:
DEFAULT_BASE_MODEL = "../MODELS/gpt2-medium-local/"            # local base model folder (saved_pretrained)
MY_MODEL_DIR = "../MODELS/Project_Mozart_gpt2-medium"        # where LoRA adapters live
CACHE_DIR = "../MODELS/"   
DEVICE = "cuda"

In [3]:
from utils import load_model_and_tokenizer, update_model_with_mask_token

update_model_with_mask_token()

model, tokenizer = load_model_and_tokenizer()

Loading tokenizer...
<MASK> token already exists in vocabulary
Loading base model with correct vocab size...


`torch_dtype` is deprecated! Use `dtype` instead!
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loading LoRA adapters...
Saving updated model...


d:\Codess & Projects\.venv\lib\site-packages\peft\utils\save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Update complete!



---

## RETRAIN

---

In [1]:
from retrain import train_lora
from utils import load_model_and_tokenizer

model, tokenizer = load_model_and_tokenizer(for_training = True)
metrics_history = train_lora(model, tokenizer)

Trainable params: 2,162,688 / 356,995,072 (0.61%)
Found 106 rest tokens


Epoch 1/25 [Train]:   0%|          | 0/218 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


KeyboardInterrupt: 

In [ ]:
from retrain import plot_metrics

plot_metrics(metrics_history)